In [1]:
!pip install torch_geometric

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv
from torch_geometric.nn import GATConv

from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.7 MB/s eta 0:00:00


Processing...
Done!


In [8]:
print("Liczba węzłów:", data.num_nodes)
print("Liczba krawędzi:", data.num_edges)
print("Liczba cech węzła:", data.num_node_features)
print("Liczba klas:", dataset.num_classes)

print("Węzły treningowe:", int(data.train_mask.sum())) #uczy się dla małej ilości, widzi cały graf, ma uogólniać na całosć
print("Węzły walidacyjne:", int(data.val_mask.sum()) if hasattr(data, 'val_mask') else "brak")
print("Węzły testowe:", int(data.test_mask.sum()))

print("Przykładowe cechy węzłów:\n", data.x[:5])
print("Etykiety:\n", data.y)

Liczba węzłów: 2708
Liczba krawędzi: 10556
Liczba cech węzła: 1433
Liczba klas: 7
Węzły treningowe: 140
Węzły walidacyjne: 500
Węzły testowe: 1000
Przykładowe cechy węzłów:
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
Etykiety:
 tensor([3, 4, 4,  ..., 3, 3, 3])


In [3]:
class GCN(torch.nn.Module): #(Graph Convolutional Network) Dwuwarstwowa sieć konwolucyjna na grafach
    def __init__(self):
        super().__init__()
        self.c1 = GCNConv(data.num_node_features, 16)  #warstwa mapująca cechy wierzchołka i jego sąsiadów (bez rozróżniania ich) do przestrzeni o wymiarze 16
        self.c2 = GCNConv(16, dataset.num_classes) #warstwa mapująca do liczby klas

    def forward(self, x, edge_index):
        x = F.relu(self.c1(x, edge_index))
        x = self.c2(x, edge_index)
        return x

class GCN3(torch.nn.Module):  #trójwarstwowa GCN by zbierać informacje z dalszego sąsiedztwa grafu
    def __init__(self):
        super().__init__()
        self.c1 = GCNConv(data.num_node_features, 16)
        self.c2 = GCNConv(16, 16)
        self.c3 = GCNConv(16, dataset.num_classes)

    def forward(self, x, edge_index):
        x = F.relu(self.c1(x, edge_index))
        x = F.relu(self.c2(x, edge_index))
        x = self.c3(x, edge_index)
        return x

class GraphSAGE(torch.nn.Module): #(Graph Sample and Aggregate) agreguje sąsiadów, potem uczy się, jak ich połączyć z wierzchołkiem
    def __init__(self):
        super().__init__()
        self.c1 = SAGEConv(data.num_node_features, 16, aggr="mean") #dla wierzchołka nauka reprezentacji dla bezpośrednich sąsiadów, agregacja za pomocą średniej
        self.c2 = SAGEConv(16, dataset.num_classes, aggr="mean")

    def forward(self, x, edge_index):
        x = F.relu(self.c1(x, edge_index))
        x = self.c2(x, edge_index)
        return x

class GAT(torch.nn.Module): #(Graph Attention Network) uczy się wag sąsiadów zamiast traktować ich jednakowo
    def __init__(self):
        super().__init__()
        self.c1 = GATConv(data.num_node_features, 8, heads=8) #8 głow (niezależnych) o wymiarze 8, ich wartości na koniec uśredniane
        self.c2 = GATConv(8 * 8, dataset.num_classes, heads=1)

    def forward(self, x, edge_index):
        x = F.elu(self.c1(x, edge_index)) #ELU (Exponential Linear Unit) x dla >0 i exp(x)-1 dla x<=0
        x = self.c2(x, edge_index)
        return x

In [4]:
def train(model, train_mask, test_mask, epochs):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    loss_fn = torch.nn.CrossEntropyLoss() #funkcja straty do klasyfikacji wieloklasowej

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad() #wyzerowanie gradientów
        forward = model(data.x, data.edge_index)  #podaje cechy i struktórę grafu, liczy logity klas węzłów treningowych (ale analizowany całego graf)
        loss = loss_fn(forward[train_mask], data.y[train_mask]) #data.y to etykiety
        loss.backward() #obliczenie gradientów funkcji straty względem parametrów modelu
        optimizer.step()  #aktualizacja parametrów

    model.eval()
    pred = forward.argmax(dim=1)[test_mask] #wybór przewidywanej klasy dla węzłów testowych
    acc = (pred == data.y[test_mask]).sum().item() / test_mask.sum().item()
    return acc

In [5]:
epochs=200

model1 = GCN()
acc1 = train(model1, data.train_mask, data.test_mask, epochs)

model2 = GCN3()
acc2 = train(model2, data.train_mask, data.test_mask, epochs)

model3 = GraphSAGE()
acc3 = train(model3, data.train_mask, data.test_mask, epochs)

model4 = GAT()
acc4 = train(model4, data.train_mask, data.test_mask, epochs)

print(f"GCN 2-warstwowy:     {acc1:.3f}")
print(f"GCN 3-warstwowy:     {acc2:.3f}")
print(f"GraphSAGE (mean):    {acc3:.3f}")
print(f"GAT (attention):    {acc4:.3f}")

GCN 2-warstwowy:     0.776
GCN 3-warstwowy:     0.754
GraphSAGE (mean):    0.775
GAT (attention):    0.765
